# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [1]:
from data_loader import MyDataLoader
%load_ext autoreload 
%autoreload 2
dl = MyDataLoader("combine")



In [2]:
dl.normalize_train_data()

In [3]:
dl.train_test_split(0.2)

In [4]:
try:
    del X_train
    del y_train
    del X_test
    del y_test
except:
    pass

X_train, y_train, X_test, y_test = dl.to_numpy_array("rgb")

In [7]:
from PIL import Image
import numpy as np

print(X_train.shape)
print(X_test.shape)
im = Image.fromarray(X_train[0], mode="YCbCr")
im.show()
print(X_test[0])

(9262, 500, 500, 3)
(9262, 500, 500, 3)
[[[ 55  45  35]
  [ 56  46  36]
  [ 56  46  36]
  ...
  [ 47  43  31]
  [ 47  43  31]
  [ 47  43  31]]

 [[ 52  42  32]
  [ 53  43  33]
  [ 54  44  34]
  ...
  [ 52  48  36]
  [ 52  48  36]
  [ 52  48  36]]

 [[ 49  39  29]
  [ 50  40  30]
  [ 51  41  31]
  ...
  [ 55  51  39]
  [ 55  51  39]
  [ 55  51  39]]

 ...

 [[146 106 107]
  [146 106 107]
  [145 105 105]
  ...
  [165 158 166]
  [165 158 166]
  [165 158 166]]

 [[158 118 119]
  [158 118 119]
  [157 117 117]
  ...
  [175 168 176]
  [175 168 176]
  [175 168 176]]

 [[165 125 126]
  [164 124 125]
  [164 124 124]
  ...
  [180 173 181]
  [180 173 181]
  [180 173 181]]]


In [8]:
from PIL import Image
import numpy as np

img = Image.open("test.jpg").convert('L')
img = img.resize((500,500))
img.show()
arr = np.array(img)

In [17]:
from PIL import Image
import numpy as np

img = Image.open("bla.jpg")
img = img.resize((500,500))
x=np.ndarray((1,500,500,3), dtype=np.short)
img0 = Image.fromarray(x)

TypeError: Cannot handle this data type: (1, 1, 500, 3), <i2